In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from typing import List
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from tqdm import tqdm
from collections import namedtuple

In [3]:
df = pd.read_hdf('../datasets/XBTUSD_1min.h5', key='XBT')

In [4]:
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = pd.to_numeric(df['High'])
df['Low'] = pd.to_numeric(df['Low'])
df['Close'] = pd.to_numeric(df['Close'])
df['Volume'] = pd.to_numeric(df['Volume'])

In [5]:
df_month = df['2019-08-1':'2019-08-2']

In [6]:
low_indx = df_month['Low'].argsort().values

In [23]:
class LongTrade():
    def __init__(self, entry_price : float, take_profit_pct : float, entry_date : datetime, leverage : int = 10):
        self.entry_price = entry_price
        self.take_profit_pct = take_profit_pct
        self.entry_date = entry_date
        self.leverage = leverage
        
        self.close_price : float =  None
        self.close_date : datetime = None
        self.profit = None
        
    def calc_exit(self, df_prices):
        tp_price = self.entry_price * (1 + self.take_profit_pct / 100)
        res = df_prices[df_prices['High'] >= tp_price]
        
        if len(res) == 0:
            return
        
        self.close_price : float = tp_price
        self.close_date : datetime = res.index[0]
        self.profit : float = self.take_profit_pct * self.leverage

In [36]:
def isTradeOverlapping(trades: List[LongTrade], trade: LongTrade):
    if len(trades) == 0:
        return False
    
    for t in trades:
        if trade.entry_date <= t.close_date and trade.entry_date >= t.entry_date:
            return True
        if trade.close_date <= t.close_date and trade.close_date >= t.entry_date:
            return True
        if trade.close_date >= t.close_date and trade.entry_date <= t.entry_date:
            return True
    return False

In [54]:
trades : List[LongTrade] = []

PROFIT_PCTG = 2
    
for low_i in tqdm(low_indx):
    trade = LongTrade(df_month['Low'].iloc[low_i], PROFIT_PCTG, df_month.index[low_i])
    trade.calc_exit(df_month.iloc[low_i+1:])
    if trade.profit is not None and not isTradeOverlapping(trades, trade):
        trades.append(trade)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2880/2880 [00:02<00:00, 1142.40it/s]


In [55]:
print(len(trades), sum([t.profit for t in trades]))

5 100


In [56]:
fig = go.Figure(data=[go.Candlestick(x=df_month.index,
                open=df_month['Open'], high=df_month['High'],
                low=df_month['Low'], close=df_month['Close'])
                     ])
for t in trades:
    fig.add_shape(type="rect",
        x0=t.entry_date, y0=t.entry_price, x1=t.close_date, y1=t.close_price,
        line=dict(
            color="RoyalBlue",
            width=2,
        )
    )

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()